### Project 02: Create a GUI Notebook Program

Samuel Nnadi 3/29/2024
Young Lee nominal change for lab 06

Project 2 will adapt the procedural code we have been working on in INST326_SimpleGUI_Note_Form_IO.ipynb to create an OOP version of the program using three classes.

    A Notebook or MainWindow class
    A Form or TopWindow class
    A Note class

The MainWindow class is a subclass of Tkinter’s tk.Tk class and thus inherits its attributes and methods, while allowing us to customize the new window objects to our needs. These new window objects will represent “notebooks” or collections of notes, and allow us to work with those notes. (I have named it MainWindow because this class definition could be used to create any kind of main window in Tkinter. We are using it to represent notebooks in this application, but you might use it for other purposes in onther applications.)


The TopWindow class creates a new top window in Tkinter, which is essentially a form for entering the title, text, links, and tags for our note. When we submit the note, this “form” object has a method that creates the note’s metadata and a new Note object. That note object is appended to the list of all notes, which is an attribute of the notebook (MainWindow) class.
The Note class creates note objects that contain the  title, text, links, tags, and metadata for each note.

For Project 02 you will:  

    1. Create one notebook or MainWindow object  
    2. Create several (at least 3) ‘real’ notes for your final submission. By ‘real’ I mean actual notes about python that are useful to you. Print them in the cell at the bottom of the notebook.
    3. Save those notes to a single .txt, .csv, or .json file (your choice of format).  
    4. Retrieve those notes and 
    5. Display representations of them as either labels or buttons in the  main window (remember how we displayed cards in project 01). These representations are not whole notes. Rather, they are object representations of the notes.  
    6. When they are clicked the whole note pops up in a new window - either the form window or a ‘read-only’ version of the form window.



#### Complete your code in the cell below

The code cell below contains the imports you will need; the top level structure for the three classes to get you started; and the execution code at the bottom. 

In [38]:
#Import
import tkinter as tk
from tkinter import ttk
from tkinter import filedialog
import datetime 

class MainWindow(tk.Tk):
    def __init__(self):  
        super().__init__() 
        
        self.geometry("600x400") 
        self.title('Notebook') 
        self.config(bg='light blue')
        self.notebook = []
        self.notes = [] 
        # To Customize the buttons, and labels
        self.style = ttk.Style()
        self.style.configure('Custom.TButton', foreground='black')
        
        # Here is where to can enter your temporary Title
        note_title_label = ttk.Label(self, text="Preview title:")
        note_title_label.grid(row=0, column=0, padx=5, pady=5)
        self.note_title = ttk.Entry(self)
        self.note_title.grid(row=0, column=1, padx=5, pady=5)

        # This is where the new notes will go after clicking the "new" button
        note_text_label = ttk.Label(self, text="Notes:")
        note_text_label.grid(row=1, column=0, padx=5, pady=5)
        self.note_text = tk.Text(self)
        self.note_text.grid(row=1, column=1, padx=5, pady=5)
        
        ## Here is where we create the buttons for the Main Window
        # This button takes you to another window where to can create a new button for your notes
        self.new_button = ttk.Button(self, text="New", command=self.new_note, style='Custom.TButton')
        self.new_button.grid(row=2, column=0, padx=5, pady=5)
        
        # Here were you can bring in notes you another saved, and look at them
        self.open_button = ttk.Button(self, text="Open", command=self.open_notebook, style='Custom.TButton')
        self.open_button.grid(row=2, column=1, padx=5, pady=5)
        
        # This is where you save all the notes you created into one file.
        self.save_button = ttk.Button(self, text="Save", command=self.save_notebook, style='Custom.TButton')
        self.save_button.grid(row=2, column=2, padx=5, pady=5)
        
        # a frame for your new buttons
        self.note_list_frame = tk.Frame(self)
        self.note_list_frame.grid(row=1, column=0, columnspan=3, padx=10, pady=10)
        self.note_buttons = []

        
        
    def new_note(self):
        note_window = NoteForm(self, self.notebook, self.update_note_list)

    # Here is the command that lets you open the buttons.
    def open_notebook(self):
        filepath = filedialog.askopenfilename(initialdir="C:\\Users\\samuelnnadijr\\Samuel INST326 Note\\Notes",
                                         filetypes=[("text files", "*.txt"), 
                                                    ("csv files", ".csv"),
                                         ("all files", "*.*")])
        if filepath:
            with open(filepath, "r") as file:
                for line in file:
                    title = line.strip()
                    text = file.readline().strip()
                    meta = file.readline().strip()
                    self.notebook.append(MakeNote({"title": title, "text": text, "meta": meta}))
                self.update_note_list()
    #Here is the command that lets you save the buttons.
    def save_notebook(self):
        filepath = filedialog.asksaveasfilename(defaultextension=".txt", filetypes=(("Text files", "*.txt"), ("All files", "*.*")))
        
        if filepath:
            with open(filepath, "w") as file:
                for note in self.notebook:
                    file.write(f"{note.title}\n{note.text}\n{note.meta}\n")
                    
            
    def update_note_list(self):
        # Clears existing buttons, it's like a reset to a gaame
        for button in self.note_buttons:
            button.destroy()
        self.note_buttons.clear()

        # Creates new buttons for each note
        for index, note in enumerate(self.notebook):
            button = ttk.Button(self.note_list_frame, text=note.title, style='Custom.TButton',
                                command=lambda i=index: self.display(i))
            button.grid(row=index, column=0, padx=5, pady=5, sticky="ew")
            self.note_buttons.append(button)
    # This helps display the notes you created
    def display(self, index):
        selected_note = self.notebook[index]
        popup = tk.Toplevel()
        popup.title(selected_note.title)
        text_wg = tk.Text(popup)
        text_wg.insert(tk.END, f"Title: {selected_note.title}\n")
        text_wg.insert(tk.END, f"Text: {selected_note.text}\n")
        text_wg.insert(tk.END, f"Meta: {selected_note.meta}\n")

        text_wg.pack()

class NoteForm(tk.Toplevel):
    
    def __init__(self, master, notebook, update_note_list): # initialize the new object
        super().__init__(master) # initialize it as a toplevel window

        self.update_note_list = update_note_list

        note_title_label = ttk.Label(self, text="Title:")
        note_title_label.grid(row=0, column=0, padx=5, pady=5)
        self.note_title = ttk.Entry(self)
        self.note_title.grid(row=0, column=1, padx=5, pady=5)

        note_text_label = ttk.Label(self, text="Text:")
        note_text_label.grid(row=1, column=0, padx=5, pady=5)
        self.note_text = tk.Text(self)
        self.note_text.grid(row=1, column=1, padx=5, pady=5)
        
        # Here when you hit submit, it saves what put in, into a button, and added to a list.
        submit_button = ttk.Button(self, text="Submit", command=self.submit, style='Custom.TButton')
        submit_button.grid(row=2, column=0, columnspan=2, padx=5, pady=5)
        

        note_title_label = ttk.Label(self, text="Links:")
        note_title_label.grid(row=3, column=0, padx=5, pady=5)
        note_link = tk.Entry(self, width=80)
        note_link.grid(padx=10, pady=10, row=3, column=1, sticky='w')
        note_link.insert(0, 'If there is a link with this note enter it here. Else, enter "None"') 

        # create our note link entry field
        note_title_label = ttk.Label(self, text="Tags :")
        note_title_label.grid(row=4, column=0, padx=5, pady=5)
        note_tags = tk.Entry(self, width=80)
        note_tags.grid(padx=10, pady=10, row=4, column=1, sticky='w')
        note_tags.insert(0, 'Enter #tags, separated by commas') 
        self.notebook = notebook
    # Here sumbits gets the title, text, and meta 
    def submit(self):
        created = datetime.datetime.now()
        local_now = created.astimezone() # shows the local time and the GMT offset
        local_tz = local_now.tzinfo 
        title = self.note_title.get()
        text = self.note_text.get("1.0", tk.END).strip()
        meta = f'created {created}, {local_tz}'
        new_note = MakeNote({"title": title, "text": text, "meta": meta})
        self.notebook.append(new_note)
        self.update_note_list()
        self.destroy()
        
#Transforms dictionary with data into a new note object
class MakeNote():
    def __init__(self, note_dict):
        self.title = note_dict["title"]
        self.text = note_dict["text"]
        self.meta = note_dict["meta"]
        

if __name__ == '__main__':
    
    main_window = MainWindow() 

    main_window.mainloop()

#### Print your three notes below

In [32]:
# print your notes here
hand = open("/Users/samuelnnadijr/Desktop/Projects/INSTProject2Notes.txt")
for pen in hand:
    print(pen)

ttk.Style()

ttk.Style() works similarly to how bg works, where it lets you customize your widgets like labels and buttons. An example of how to use them is for first do, style = ttk.Style(), then under it put down, style.configure('Custom.TButton', foreground='blue'), with this it will make the texts in your button/label blue.



style = ttk.Style(),



style.configure('Custom.TButton', foreground='blue')



new_button = ttk.Button(self, text="New", command=new_note, style='Custom.TButton')

new_button.grid(row=2, column=0, padx=5, pady=5)

created 2024-03-31 21:01:09.548688, EDT





Creating new Buttons

To create new buttons, you have to create a loop where when a condition is done or a loop that automatic does it, the button will be created. For example, the condition could be when you click on this button a new button will be created. 



Example,

import tkinter as tk

from tkinter import ttk



window = tk.Tk()

for i in range(5):

    button = ttk.Button(window, text="1")

   